In [1]:
import os
import requests
import datetime
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
options = Options()
browser = Chrome(ChromeDriverManager().install(), options=options)

/var/folders/74/n55gd6gs38n5wj2k6hwrc_yc0000gn/T/ipykernel_46043/748855775.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome(ChromeDriverManager().install(), options=options)


## Collect All Reviews for a Reviewer

In [12]:
userhandle = "alamakgirl" ### TO CHANGE HERE
URL = "https://www.burpple.com/@" + userhandle + "/timeline"

In [13]:
browser.get(URL)

In [14]:
load_more = True
while load_more:
    try:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        more_reviews = browser.find_element("id", "masonryViewMore-btn")
        time.sleep(1)
        more_reviews.click()
        time.sleep(1)
    except Exception as e:
        load_more = False
        print(e)
        

In [15]:
# find all reviews by a given revier
page_html = BeautifulSoup(browser.page_source, "html.parser")
all_reviews = page_html.findAll(class_="food card feed-item")

In [16]:
reviews_by_reviewer = []
for review_listing in all_reviews:
    # name and address
    header = review_listing.find(class_="food-place card-item card-item--header")
    try: 
        name = header.find(class_="card-item-set--link-title").text
        name = name.replace('\n', '')

        address = review_listing.find(class_="card-item-set--link-subtitle").text
        address = address.replace('\n', '')
    except:
        name = None
        address = None

    # review
    review = review_listing.find(class_="food-description").text

    # date
    user_card = review_listing.find(class_="food-user card-item")
    try:
        date = user_card.find(class_="card-item-set--link-subtitle").text
        date = date.replace('\n', '')
    except:
        date = None

    # restaurant listing link
    link = review_listing.find_all("a", href=True)[0]['href']
    link = 'https://www.burpple.com' + link

    reviews_by_reviewer.append([name, address, review, date, link])

In [17]:
reviews_by_reviewer_df = pd.DataFrame(reviews_by_reviewer, columns=['name', 'address', 'review', 'date', 'link'])

In [18]:
reviews_by_reviewer_df

,name,address,review,date,link
0,Ya Kun Kaya Toast,"2 Tampines Central 5, Singapore",\nBreakfast \nSet A (S$5.60)\nKaya butter toas...,1h ago,https://www.burpple.com/ya-kun-kaya-toast-66?b...
1,511 Lor Mee,"509 Bedok North Street 3, Singapore",\nLunch\nLor Mee (S$4.50)\nKueh tiao topped wi...,3h ago,https://www.burpple.com/511-lor-mee?bp_ref=%2F...
2,McDonald's,"60 Yishun Avenue 4, Yishun",\nSupper\nQuarter Pounder with Cheese Meal (S$...,18h ago,https://www.burpple.com/mcdonalds-555?bp_ref=%...
3,Tian Yu Tian Fish Head Steamboat,"537 Bedok North Street 3, Singapore",\nZi Char\nCrispy Noodles (S$4.50)\n,1d ago,https://www.burpple.com/tian-yu-tian-fish-head...
4,Kiroi Freshly Baked Cheese Cake,"123 Bukit Merah Lane 1, Singapore",\nCake\nOriginal (S$9)\nSoft and pillowy. Japa...,1d ago,https://www.burpple.com/kiroi-freshly-baked-ch...
...,...,...,...,...,...
67,Traditional Haig Road Putu Pirin...,"930 Yishun Avenue 2, Singapore ...",\nSeasonal Item\nRed Velvet & Nutella Putu Pir...,Feb 13 at 11:17am,https://www.burpple.com/haig-road-putu-piring?...
68,Sushi-GO (AMK Hub),"53 Ang Mo Kio Avenue 3, Singapor...",\nBirthday Treat\n<>\nDouble stamp during birt...,Feb 13 at 9:40am,https://www.burpple.com/sushi-go-amk-hub?bp_re...
69,Sweetlands Bread & Bakery ...,"10/12 Kim Keat Lane, Singapore ...",\nCake\nCempedak Cake (S$15)\nPrice increased ...,Feb 12 at 11:39pm,https://www.burpple.com/sweetlands-confectiona...
70,The Coffee Bean & Tea Leaf (Nort...,"1 Northpoint Drive, Singapore ...",\nBirthday Treat\n<>\nFree slice of cake\nFor ...,Feb 12 at 11:25pm,https://www.burpple.com/the-coffee-bean-tea-le...


In [ ]:
reviews_by_reviewer_df.to_csv("reviewer-data/" + userhandle + "_reviews.csv")